In [121]:
%load_ext autoreload
%autoreload 2

from src.cleaner.auto_cleaner import clean_text,ml_clean_text
import glob
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# cleaningの検証

In [122]:

from src.cleaner.hojichar_filter import hoji_filter, prob_hoji_filter, prob_filter
#クリーニングする処理
text="衝撃!!|中山トンネルは、上越新幹線高崎駅 - 上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。"
#text="求人情報求人情報求人情報求人情報求人情報は､､aaaaaaaaaaaaaaaaa"
#text="中山トンネルは、上越新幹線高崎駅 - 上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。"
print(clean_text(text))
#print(prob_filter(text))

中山トンネルは、上越新幹線高崎駅-上毛高原駅間にある総延長14,857 mの複線鉄道トンネルであり高崎方面から進行すると榛名トンネルの次、二番目に通過するトンネルである。建設中に2回の大出水事故を起こして難工事を極め、2回の経路変更によりようやく完成したが、経路変更のためにトンネル内に半径1,500 mの曲線ができてしまい、営業速度240 km/hの新幹線がトンネル内の曲線部分を通過するときには160 km/hに減速せざるをえなくなった。


In [127]:
#以下､クラスタリング後のjsonlファイルを読み込む例｡
#datasetsライブラリから､適当にmc4-jaなどを読み込んで問題ナシ｡
path_list=glob.glob("../data/dedup_categorized/**/*.jsonl",recursive=True)


In [128]:
len(path_list)

10989

In [129]:
random.shuffle(path_list)

In [1]:

from src.classify.Text2Vec import Text2Vec, texts2classes
from gensim.models.fasttext import load_facebook_model
t2v = Text2Vec(load_facebook_model('../data/model/entity_vector/entity_vector.model.bin'))

FileNotFoundError: [Errno 2] No such file or directory: '../data/model/entity_vector/entity_vector.model.bin'

In [3]:
from gensim.models import KeyedVectors
model_dir = '../data/model/entity_vector/entity_vector.model.bin'
model = KeyedVectors.load_word2vec_format(model_dir, binary=True)

In [6]:
model.get_vector("あ")

array([-0.3196374 ,  1.8061428 , -1.6604292 , -0.52288646, -1.0512975 ,
        1.5391749 , -1.1566727 , -0.82461774, -0.35898477,  1.2454671 ,
       -1.3672171 ,  2.7784317 ,  1.9350111 , -1.0831355 , -0.68428457,
       -2.2910554 ,  1.3848027 , -2.5809252 ,  1.7140344 , -0.02286446,
        0.48743463, -3.0924594 ,  1.6131709 , -2.3386939 ,  0.9431879 ,
       -1.3243551 , -1.5037524 , -0.58501345, -0.59000933, -1.2175318 ,
        0.5918976 ,  0.8936033 ,  2.92767   ,  2.5808516 ,  0.25240132,
        1.3072065 , -0.99929696, -0.65497893,  0.7952702 ,  0.5303069 ,
        0.7305566 , -1.6341536 ,  3.4560654 ,  0.02341504,  1.2947271 ,
        0.8061331 ,  0.1369439 , -1.1958709 ,  0.5880644 ,  0.9518098 ,
       -1.5347742 , -0.03704506,  1.9196514 , -0.49741378,  0.18996468,
        2.0485742 , -1.4030429 , -0.7410698 ,  0.91882706,  2.25174   ,
       -1.2315228 ,  1.6947681 , -1.0058278 , -2.6799057 , -0.7104148 ,
        3.435787  , -1.4046692 ,  0.45291036,  1.0565063 , -0.84

: 

In [130]:
line_list=[]
path=random.choice(path_list)
with open(path,"r") as f:
    #1行ずつ読み込む
    for line in f:
        text=json.loads(line)["text"]
        cleaned=clean_text(text)
        print(cleaned)
        break

仲間割れをしたアベンジャーズたちの空港でのバトルが続いている。キャプテン・アメリカとホークアイ対、スパイダーマンとヴィジョンの対決だ。スーパージャンパーを使ってヒーローたちを空高く飛ばしてトラックの火を消したり、オイル缶を爆発させてキャプテン・アメリカの攻撃を止めよう。スパイダーマンのウェブブラストで、バイクに乗って突進してくるホークアイから身を守れ。


In [192]:
text_list=[]
cnt=0

while True:
    path=random.choice(path_list)
    with open(path,"r") as f:
        #1行ずつ読み込む
        for line in f:
            cnt+=1
            text=json.loads(line)["text"]
            text_list.append(text)
    if cnt>100:
        break
random.shuffle(text_list)
text_list=text_list[:10]

In [193]:
from tqdm import tqdm
cleaned_list=[ml_clean_text(text) for text in tqdm(text_list)]
cleaned_list[:10]

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 233.70it/s]

0.9955269694328308
0.9958707690238953
0.7076561450958252
0.6594078540802002
0.5173245668411255
0.6453263759613037
0.983125627040863
0.6897611021995544
0.991079568862915
0.546472430229187


['',
 '',
 '似顔絵のコツを知りたいという方。それからやさしい雰囲気の似顔絵を描きたいという方の役に立てる内容にしていきたいと思います。BIRDFULの似顔絵は「やさしさ」をテーマにして描いています。',
 '森田健作ではなかったの!',
 'プーケット行きの乗り継ぎチケットも羽田で発券済なので、飛行機から降りたらまずtransfer counterを目指して進みます。transfer counterはWESTとEASTの両サイドにあって迷いましたが、スワンナプーム空港の国内線はコンコースA,BのあるEASTへ進みます。\n入国審査を通過する際にシールを渡され胸元に貼るように指示されます。これはプーケットに着いた際に手荷物の受け取り場所が国内線用と国際線用の2つに分かれているので間違えないよう注意するシールのようです。日本から預けた私たちの荷物は国際線用の手荷物カウンターに出てくるので、プーケットに着いたらこのシールと同じ表記の場所で受け取ります。おそらくこれをつけていると間違えそうになった場合現地のスタッフが案内をしてくれるのだと思います。\nそれにしてもバンコクの空港はエアコンもよく効いていて寒いです。コートは置いてきたとはいえ冬服のセーターを着ているのに寒いと感じるのには少しビックリでした。次行くことがあればブランケットや羽織るものがあるとよさそうです。',
 'シェークスピアの史劇「ヘンリー5世」にちなみ、ボドキン(矢じりの古語)の名を冠した気鋭のマイクロ・ワイナリー。醸造家のクリストファー・クリステンセンはスタンフォード大学を卒業後、20代の若さでメドロック・エイムズにて醸造家として活躍する傍ら、2011年より自らのブランドで繊細な食事にもマッチするエレガントなスタイルのソーヴィニヨン・ブランを造り始めた。探究心の強い彼はソーヴィニヨン・ミュスケ単独のワインや赤ワインのように果皮を付けたまま発酵させる古のスタイルなどにも挑戦、個性的で素晴らしい味わいに仕上げている。またシャルマ方式のスパークリングなどにもチャレンジ、その個性的な味わいで最も人気のある一本となっている。繊細な風味を愛する彼は独特の食文化を持つ日本にも深い興味を持っており日本への進出を喜んでいる。\n価格が安く質の良いぶどうを手に入れることに長けたクリステンセンはノース・コースト各地

In [194]:

for line in cleaned_list:
    
    t=line[:30]
    if t!="":
        print(t[:30])
    t=line[-30:]
    if t!="":
        print(t[-30:])

似顔絵のコツを知りたいという方。それからやさしい雰囲気の似顔
RDFULの似顔絵は「やさしさ」をテーマにして描いています。
森田健作ではなかったの!
森田健作ではなかったの!
プーケット行きの乗り継ぎチケットも羽田で発券済なので、飛行機
くことがあればブランケットや羽織るものがあるとよさそうです。
シェークスピアの史劇「ヘンリー5世」にちなみ、ボドキン(矢じ
スパイスの風味が食欲をそそるコストパフォーマンス抜群の1本。
もうすぐ彼や旦那さんの誕生日。やっぱり身につけて欲しいからお
占めるスーツ姿。バッチリ似合うお財布をここで教えちゃいます!


In [189]:
cleaned_list=[clean_text(text) for text in tqdm(text_list)]
cleaned_list[:10]

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 88.59it/s]


['夢喰いメリー第12話「夢魘」最終回と思っていたら実はまだもう1話残っていたんですねっ汗そして、本当にどこまで行っても夢もキボーも見えてこないシリアスさですっ\n一緒に戻ってきた千鶴が突然牢獄のデイドリームへいざなう。そして彼女の夢魔レオンが現れる。二人はミストルティンについて「レオンの銃でなければ、ミストルティンは倒せない」と語り始める。先生斬ればよくね?最終回だと勝手に思ってたけど実は違った第12話。転校生とその夢魔から、ミストルティンを倒すには彼らが持ってる銃が必要と言われますが、それがまた、夢もキボーもありゃしませんよねっ汗弾を作るためにあと一人、犠牲にしなくちゃいけない、それは嫌でも、そうするとミストルティンによってどんどん犠牲者が増えてしまう・・・大を活かすために小を殺すか、小を助けるために大を晒すか、はもしかしたら永遠に全員が納得する答えが出ない身近な選択かもしれませんね。しかもその小が知り合いならなおさら・・・まったく、どこまで行っても夢もキボーも未だに見出せず、むしろますます絶望の淵へ、困難の淵へと落とされていきますね・・・なんとシリアス過ぎる作品なんだろう・・・・・・ってことで、戦っても勝てない、このままじゃ勇魚以前に光凪さんのキボーも破壊されてしまうので、じぶんは夢路がそのソードでミストルティンではなく飯島を斬り捨て殺っちまうことを提言しますwそうすれば器を失ったミストルティンは・・・あれ?器が死んだらどうなるんだろう?ま、まぁ、とりあえず真っ向勝負よりは現段階、最も効果がありそうなので、あと、アイツウザイのでとっとと斬り捨ててくれると嬉しいんですけどもね☆次回最終回・・・ですね。\n夢喰いメリー第12話「夢魘」\nミストルティンに完全敗北だったメリーたち。\n勇魚の夢を消すまでの猶予期間をもらったが...逆転の手はあるのか?\n千鶴の夢魔・レオンが夢路たちをデイドリームに招き。\n一人の人の夢を守るため、自らの命をさし出してでも叶えたい夢がある。夢を守ることが夢。そんな夢でいいのか?|所詮、すべては戯言なんだよ|2011/04/03 01:04|夢喰いメリー第12話「夢魘」感想\nミストルティンとのバトルからあけて光凪医院に戻ってきた夢路たち。\n一緒に戻ってきた千鶴が突然牢獄のデイドリームへいざなう。\nそして彼女の夢魔レオンが現れる。\

In [ ]:
out_path="../data/test_articles.txt"
with open(out_path,"w") as f:
    f.write("")
for path in path_list:
    with open(path,"r") as f:
        #1行ずつ読み込む
        cnt=0
        for line in f:
            cleaned=json.loads(line)["text"]
            cleaned=cleaned.replace("\n","")
            if len(cleaned)<100:
                with open(out_path,"a") as f:
                    f.write(cleaned+"\n")
            else:
                with open(out_path,"a") as f:
                    f.write(cleaned[:100]+"\n")
                with open(out_path,"a") as f:
                    f.write(cleaned[-100:]+"\n")
            cnt+=1
            if cnt>1:
                break
        

In [ ]:
random.random()

In [ ]:
print(clean_text(text))

In [ ]:
print(text)

# データ処理の作業練習

In [ ]:
import glob
# %%

pq_list=glob.glob("/data/hatakeyama/python/llm_corpus_original/**/*.parquet",recursive=True)

pq_list=[i for i in pq_list if i.endswith('.parquet')]
print(len(pq_list), " files found")



In [ ]:

import pandas as pd
for database_path in pq_list:
    #database_path="/data/hatakeyama/python/llm_corpus_original/CC-MAIN-2023-50_v2/text/batch10/CC-MAIN-2023-50-batch10-iter10404.parquet"
    df=pd.read_parquet(database_path)
    if df.shape[0]==0:
        continue
    lines=df['text'].tolist()
    break

lines
#lines=[]
#for article in read_gzip_json_file(database_path):
#    text = article.get('text', '')  # 'text'キーからテキストデータを取得
#    lines.append(text)



In [199]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from src.classify.TfidfClassifier import prepare_pileline

# テキストデータセット (仮のもの)
texts = [
    "猫は可愛い",
    "犬も可愛い",
    "猫は動物",
    "犬は友達",
    "動物園には猫がいない",
    "動物園には犬がいない",
    "可愛い犬と遊ぶ",
    "可愛い猫と遊ぶ"
]

pipe=prepare_pileline()
pipe.fit(texts)
clusters = pipe.named_steps['cluster'].labels_


clusters

array([0, 1, 0, 1, 2, 2, 1, 0], dtype=int32)

In [2]:
import joblib

kmeans = joblib.load("../data/model/kmeans.pkl")
kmeans

Pipeline(steps=[('truncator', TextTruncator()),
                ('tfidf',
                 TfidfVectorizer(norm=None,
                                 tokenizer=<function extract_nouns at 0x14c393de4400>)),
                ('cluster', MiniBatchKMeans(n_clusters=20, random_state=0))])

: 

In [114]:

# %%
import glob
import sys
import os
from concurrent.futures import ThreadPoolExecutor
import argparse
import random

# %%
with open("temp/gz_list.txt", "r") as f:
    gz_list = f.read().splitlines()

# gz_list = [i for i in gz_list if i.endswith('.gz')]
print(len(gz_list), " files found")

random.shuffle(gz_list)


# %%
import argparse
import json
import os
from src.classify.Text2Vec import Text2Vec, texts2classes
from src.cleaner.auto_cleaner import clean_text, ml_clean_text
from gensim.models.fasttext import load_facebook_model
import joblib
from src.load_gz import read_gzip_json_file, load_gzip_or_parquet
from src.distribute_jsonl import process_lines, make_dir
import pandas as pd

streaming = True
base_dir = "../data/categorized"
length_threshold = 30  # 短い記事は捨てる
check_length = 200  # はじめのlengthだけで分類する

# 機械学習で記事を選別する
do_ml_clean = True

# load models
#t2v = Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))
t2v = Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))
kmeans = joblib.load("../data/model/kmeans.pkl")


make_dir(base_dir)


def proc(docs, base_dir, database_path,
         check_length=check_length):
    return process_lines(docs, t2v, kmeans, base_dir, database_path, check_length=check_length)

# %%


# %%
batch_size = 100


def main():

    docs = []

    # gzの場合
    lines = load_gzip_or_parquet(database_path)

    for text in lines:
        if do_ml_clean:
            text = ml_clean_text(text)
        else:
            text = clean_text(text)
        if len(text) < length_threshold:
            continue

        docs.append(text)
        if len(docs) == batch_size:
            proc(docs, base_dir, database_path,
                 check_length=check_length)

            # docsをリセット
            docs = []


if __name__ == "__main__":
    main()



63755  files found


NameError: name 'database_path' is not defined

In [116]:
# gzの場合
database_path="../data/original_dump/jap2010/jap2010_0001_0.jsonl.gz"
lines = load_gzip_or_parquet(database_path)


In [117]:
docs=[]
# gzの場合
lines = load_gzip_or_parquet(database_path)

for text in lines:
    if do_ml_clean:
        text = ml_clean_text(text)
    else:
        text = clean_text(text)
    if len(text) < length_threshold:
        continue

    docs.append(text)
    if len(docs) == batch_size:
        proc(docs, base_dir, database_path,
                check_length=check_length)

KeyboardInterrupt: 

In [119]:
length=200
import numpy as np
target_texts =docs
target_texts = [i[:length] for i in target_texts]
vec = np.array([np.array(t2v.text2vec(i), dtype="float32") for i in target_texts])
classes = kmeans.predict(vec.astype("float64"))

In [120]:
classes

array([2841, 1990, 6089, 5703, 2597, 4077,   60, 2501, 8525, 7998, 6457,
       4461, 9651,  869, 1932,  108, 8631], dtype=int32)

In [ ]:
for v in vec:
    if v.dtype is not vec[1].dtype:
        print("dtype error")

In [ ]:
vec.dtype